In [93]:
using Random
using DataFrames
using CSV
using Statistics

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 400
const MXV = 40
const MNV = 2

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int)::Fighter
    if c==0
        return random_fighter()
    end
    a = random_tournament_winner(c-1)
    b = random_tournament_winner(c-1)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Fighter}, bs::Array{Fighter})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

eval_team_battle (generic function with 1 method)

In [2]:
function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [3]:
library = Array{Fighter}(undef, 100000)
for i in 1:100000
    library[i] = rand_rename(random_fighter())
end

In [4]:
function evaluate_generator(f::Function, team_size::Int, n_times::Int = 1)::AbstractFloat
    scores = Array{AbstractFloat}(undef, n_times)
    for i0 in 1:n_times
        team = Array{Fighter}(undef, team_size)
        for i in 1:team_size
            team[i] = f()
        end
        bestscore = 999999
        exploiter = library[1]
        for i in 1:length(library)
            fighter = library[i]
            score = 0
            for j in 1:team_size
                score = score + eval_battle(team[j], fighter)
            end
            if score < bestscore
                bestscore = score
                exploiter = library[i]
            end
        end
        if n_times < 5
            println(exploiter)        
        end
        scores[i0] = bestscore/team_size
    end
    return mean(scores)
end

evaluate_generator (generic function with 2 methods)

In [81]:
# new random generation to more closely approximate Nash
function random_scheme_a(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    budget_sofar = budget + 1
    while budget_sofar > budget
        hf_ratio = rand()
        l_ratio = rand()
        h_temp = rand() + 0.2
        f_temp = rand() + 0.2
        while abs(h_temp - f_temp) < 0.2
            h_temp = rand() + 0.2
            f_temp = rand() + 0.2
        end
        budget_hf = (0.1 + hf_ratio * 0.8) * (budget * .7)
        h_ratio = h_temp/sqrt(h_temp * f_temp)
        f_ratio = f_temp/sqrt(h_temp * f_temp)
        h = max(h0, round(sqrt(budget_hf) * h_ratio))
        f = max(f0, round(sqrt(budget_hf) * f_ratio))
        budget_hf = h * f
        budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
        l_ratio2 = h^2/(h^2+f^2)
        #l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
        lraw = l0 + ( budget_remain * l_ratio2 )/h
        traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
        l = trunc(lraw)
        t = trunc(traw)
        budget_sofar = h * f + h * l + f * t
        if (budget - budget_sofar) >= min(h, f)
            could_add_l = (budget - budget_sofar) >= h
            could_add_t = (budget - budget_sofar) >= f
            if could_add_l && !could_add_t
                l = l+trunc((budget - budget_sofar)/h)
            end
            if !could_add_l && could_add_t
                t = t+trunc((budget - budget_sofar)/f)
            end
            if could_add_l && could_add_t
                if fpart(lraw) > fpart(traw)
                    l = l+trunc((budget - budget_sofar)/h)
                else
                    t = t+trunc((budget - budget_sofar)/f)
                end
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)  
    return Fighter("No Name [Scheme A]", h, f, l, t)
end



random_scheme_a (generic function with 6 methods)

In [82]:
budget = MXS
h0 = MNV
f0 = MNV
l0 = MNV
t0 = MNV
hf_ratio0 = 0
l_ratio0 = 0
h_temp0 = 0
f_temp0 = 0

hf_ratio = rand()
l_ratio = rand()
h_temp = rand()
f_temp = rand()
budget_hf = (0.1 + hf_ratio * 0.8) * (budget * .7)
h_ratio = h_temp/sqrt(h_temp * f_temp)
f_ratio = f_temp/sqrt(h_temp * f_temp)
h = max(h0, round(sqrt(budget_hf) * h_ratio))
f = max(f0, round(sqrt(budget_hf) * f_ratio))
budget_hf = h * f
budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
l_ratio2 = h^2/(h^2+f^2)
#l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
lraw = l0 + ( budget_remain * l_ratio2 )/h
traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
l = trunc(lraw)
t = trunc(traw)
budget_sofar = h * f + h * l + f * t
if (budget - budget_sofar) >= min(h, f)
    could_add_l = (budget - budget_sofar) >= h
    could_add_t = (budget - budget_sofar) >= f
    if could_add_l && !could_add_t
        l = l+trunc((budget - budget_sofar)/h)
    end
    if !could_add_l && could_add_t
        t = t+trunc((budget - budget_sofar)/f)
    end
    if could_add_l && could_add_t
        if fpart(lraw) > fpart(traw)
            l = l+trunc((budget - budget_sofar)/h)
        else
            t = t+trunc((budget - budget_sofar)/f)
        end
    end
end

println((h, f, l, t, budget_hf, budget_sofar, hf_ratio))

(22.0, 5.0, 12.0, 5.0, 110.0, 394.0, 0.39016548918676763)


In [83]:
# budget = MXS
# h0 = MNV
# f0 = MNV
# l0 = MNV
# t0 = MNV
# hf_ratio0 = 0
# l_ratio0 = 0
# h_temp0 = 0
# f_temp0 = 0


# for i in 1:1000
#     hf_ratio = rand()
#     l_ratio = 0.25 + 0.5 * rand()
#     h_temp = rand() + 0.1
#     f_temp = rand() + 0.1
#     budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
#     h_ratio = h_temp/sqrt(h_temp * f_temp)
#     f_ratio = f_temp/sqrt(h_temp * f_temp)
#     h = max(h0, round(sqrt(budget_hf) * h_ratio))
#     f = max(f0, round(sqrt(budget_hf) * f_ratio))
#     budget_hf = h * f
#     budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
#     lraw = l0 + ( budget_remain * l_ratio )/h
#     traw = t0 + ( budget_remain * (1 - l_ratio) )/f
#     l = trunc(lraw)
#     t = trunc(traw)
#     budget_sofar = h * f + h * l + f * t
#     if (budget - budget_sofar) >= min(h, f)
#         could_add_l = (budget - budget_sofar) >= h
#         could_add_t = (budget - budget_sofar) >= f
#         if could_add_l && !could_add_t
#             l = l+trunc((budget - budget_sofar)/h)
#         end
#         if !could_add_l && could_add_t
#             t = t+trunc((budget - budget_sofar)/f)
#         end
#         if could_add_l && could_add_t
#             if fpart(lraw) > fpart(traw)
#                 l = l+trunc((budget - budget_sofar)/h)
#             else
#                 t = t+trunc((budget - budget_sofar)/f)
#             end
#         end
#     end

#     budget_sofar = h * f + h * l + f * t
#     if (budget - budget_sofar) >= min(h, f)
#         hf_ratio0 = hf_ratio
#         l_ratio0 = l_ratio
#         h_temp0 = h_temp
#         f_temp0 = f_temp
#         println((h, f, l, t, budget_hf, budget_sofar))
#     end
# end

In [84]:
# hf_ratio = hf_ratio0
# l_ratio = l_ratio0
# h_temp = h_temp0
# f_temp = f_temp0

In [85]:
# budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
# h_ratio = h_temp/sqrt(h_temp * f_temp)
# f_ratio = f_temp/sqrt(h_temp * f_temp)
# h = max(h0, round(sqrt(budget_hf) * h_ratio))
# f = max(f0, round(sqrt(budget_hf) * f_ratio))
# budget_hf = h * f
# budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
# lraw = l0 + ( budget_remain * l_ratio )/h
# traw = t0 + ( budget_remain * (1 - l_ratio) )/f
# l = trunc(lraw)
# t = trunc(traw)
# budget_sofar = h * f + h * l + f * t
# println((h, f, l, t, budget_hf, budget_sofar))

In [86]:
# if (budget - budget_sofar) >= min(h, f)
#     could_add_l = (budget - budget_sofar) >= h
#     could_add_t = (budget - budget_sofar) >= f
#     if could_add_l && !could_add_t
#         l = l+1
#     end
#     if !could_add_l && could_add_t
#         t = t+1
#     end
#     if could_add_l && could_add_t
#         if fpart(lraw) > fpart(traw)
#             l = l+1
#         else
#             t = t+1
#         end
#     end
# end

# budget_sofar = h * f + h * l + f * t
# println((h, f, l, t, budget_hf, budget_sofar))

## Testing Random Scheme A

In [87]:
f = random_scheme_a()
print(f)
f.h * f.f + f.h * f.l + f.f * f.t

Fighter("No Name [Scheme A]", 10, 15, 8, 11)

395

In [88]:
rt = random_team(() -> rand_rename(random_scheme_a()), 100)
fighters_to_df(rt)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Helicopter Kiaansh,11,14,9,10
2,Scissors Celio,11,6,23,13
3,Keno the Monkey,17,12,7,6
4,Primordial Mouse,18,14,5,4
5,Noble Cat,20,4,15,5
6,Airplane Cayetano,21,10,7,4
7,Sassy Giraffe,9,16,7,12
8,Xray Asim,7,23,4,9
9,Nuhamin the Underwater,28,7,6,5


In [144]:
counters = random_team(() -> pick_best_rdmly(library, rt, 9), 100)
fighters_to_df(counters)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Egotistical Icicle,9,15,7,13
2,Zinc Unicorn,14,9,12,11
3,Scorpion Reve,16,13,8,4
4,Solymar the Neurotic,13,11,7,15
5,Old Scotlynd,8,18,9,10
6,Kiernan the Jacket,16,9,14,3
7,Boisterous Beetle,11,13,13,8
8,Zesty Marmot,11,12,15,8
9,Dancing Khamil,19,7,13,2


In [90]:
for f in rt
    v = eval_battle_list(f, counters)
    if v < -25
        println(f)
        println(v)
    end
end

Fighter("Helicopter Kiaansh", 11, 14, 9, 10)
-26
Fighter("Mccall the Artisan", 11, 14, 9, 10)
-26


In [91]:
for f in rt
    v = eval_battle_list(f, counters)
    if v > 10
        println(f)
        println(v)
    end
end

Fighter("Noble Cat", 20, 4, 15, 5)
14
Fighter("Xray Remington", 5, 19, 7, 14)
26
Fighter("Jakyrie the Lawyer", 5, 22, 5, 12)
22


In [79]:
evaluate_generator(random_fighter, 100, 20)

-0.5489999999999999

In [92]:
evaluate_generator(random_scheme_a, 100, 20)

-0.485

In [33]:
evaluate_generator(() -> random_tournament_winner(1), 100, 20)

-0.5015000000000002

In [34]:
evaluate_generator(() -> random_tournament_winner(2), 100, 20)

-0.518

In [35]:
evaluate_generator(() -> random_tournament_winner(3), 100, 20)

-0.5145000000000002

In [36]:
evaluate_generator(() -> random_tournament_winner(4), 100, 20)

-0.517

## Team battles

In [123]:
team_a = random_team(() -> rand_rename(random_scheme_a()), 100)
team_r0 = random_team(() -> rand_rename(random_tournament_winner(0)), 100)
team_r1 = random_team(() -> rand_rename(random_tournament_winner(1)), 100)
team_r2 = random_team(() -> rand_rename(random_tournament_winner(2)), 100)


100-element Vector{Fighter}:
 Fighter("Balanced Domino", 23, 12, 2, 6)
 Fighter("Koala Maxi", 9, 3, 32, 28)
 Fighter("Kooky Leopard", 17, 8, 10, 11)
 Fighter("Josi the Lawyer", 9, 26, 9, 3)
 Fighter("Anneli the Pangolin", 21, 5, 10, 17)
 Fighter("Uber Warrior", 2, 13, 5, 28)
 Fighter("Junior Vacuum", 4, 27, 19, 8)
 Fighter("Izela the Vacuum", 5, 14, 7, 21)
 Fighter("Elevated Pangolin", 12, 13, 4, 15)
 Fighter("Butler Avrian", 24, 12, 2, 5)
 Fighter("Bear Yuva", 9, 13, 2, 20)
 Fighter("Mackenize the Boisterous", 4, 15, 10, 20)
 Fighter("Drayven the Notebook", 11, 17, 11, 5)
 ⋮
 Fighter("Xray Baltazar", 10, 13, 7, 15)
 Fighter("Koda the Elephant", 6, 9, 18, 26)
 Fighter("Unicorn Shanelly", 4, 12, 34, 18)
 Fighter("Happy Dylann", 3, 34, 31, 6)
 Fighter("Edvin the Hyena", 7, 14, 5, 19)
 Fighter("Jayjay the Bagel", 12, 6, 25, 4)
 Fighter("Itinerant Sombrero", 10, 10, 21, 9)
 Fighter("Aylene the Frog", 16, 16, 5, 4)
 Fighter("Victor the Ostrich", 14, 13, 3, 13)
 Fighter("Ingenious Engine", 1

In [145]:
eval_team_battle(team_a, counters)

-1

In [146]:
eval_team_battle(team_a, team_r0)

1

In [147]:
eval_team_battle(team_a, team_r1)

1

In [148]:
eval_team_battle(team_a, team_r2)

1

In [149]:
eval_team_battle(team_r0, team_a)

-1

In [150]:
eval_team_battle(team_r0, team_r1)

1

In [151]:
eval_team_battle(team_r0, team_r2)

-1

In [158]:
as = team_r2
bs = counters
a_i = 1
b_i = 1
print("         ")
println(as[a_i])
print("         ")
println(bs[b_i])
while (a_i <= length(as)) && (b_i <= length(bs))
    res = eval_battle(as[a_i], bs[b_i])
    if res == 1
        println(string(as[a_i].name, " won against ", bs[b_i].name))
        b_i = b_i + 1
        print("         ")
        println(bs[b_i])
    else
        if res == 0
            println(string(as[a_i].name, " tied against ", bs[b_i].name))
            b_i = b_i + 1
            print("         ")
            println(bs[b_i])
        else
            println(string(as[a_i].name, " lost against ", bs[b_i].name))
        end
        a_i = a_i + 1
        print("         ")
        println(as[a_i])
    end
end
a_out = (a_i > length(as))
b_out = (b_i > length(as))
if a_out
    if b_out
        return 0
    else
        return -1
    end
else
    return 1
end


         Fighter("Balanced Domino", 23, 12, 2, 6)
         Fighter("Egotistical Icicle", 9, 15, 7, 13)
Balanced Domino won against Egotistical Icicle
         Fighter("Zinc Unicorn", 14, 9, 12, 11)
Balanced Domino lost against Zinc Unicorn
         Fighter("Koala Maxi", 9, 3, 32, 28)
Koala Maxi lost against Zinc Unicorn
         Fighter("Kooky Leopard", 17, 8, 10, 11)
Kooky Leopard lost against Zinc Unicorn
         Fighter("Josi the Lawyer", 9, 26, 9, 3)
Josi the Lawyer lost against Zinc Unicorn
         Fighter("Anneli the Pangolin", 21, 5, 10, 17)
Anneli the Pangolin lost against Zinc Unicorn
         Fighter("Uber Warrior", 2, 13, 5, 28)
Uber Warrior lost against Zinc Unicorn
         Fighter("Junior Vacuum", 4, 27, 19, 8)
Junior Vacuum lost against Zinc Unicorn
         Fighter("Izela the Vacuum", 5, 14, 7, 21)
Izela the Vacuum won against Zinc Unicorn
         Fighter("Scorpion Reve", 16, 13, 8, 4)
Izela the Vacuum lost against Scorpion Reve
         Fighter("Elevated Pangolin", 

         Fighter("Carmello the Rogue", 17, 2, 19, 21)
Shy Marmot lost against Carmello the Rogue
         Fighter("Brutal Talyn", 23, 4, 8, 31)
Brutal Talyn lost against Carmello the Rogue
         Fighter("Valiant Horse", 17, 5, 14, 15)
Valiant Horse lost against Carmello the Rogue
         Fighter("Large Monkey", 15, 6, 12, 21)
Large Monkey lost against Carmello the Rogue
         Fighter("Naughty Robot", 11, 22, 2, 6)
Naughty Robot won against Carmello the Rogue
         Fighter("Xquisite Cake", 13, 20, 7, 2)
Naughty Robot won against Xquisite Cake
         Fighter("Yogurt Stormy", 25, 5, 10, 5)
Naughty Robot lost against Yogurt Stormy
         Fighter("Militant Quilt", 12, 16, 2, 11)
Militant Quilt lost against Yogurt Stormy
         Fighter("Ruaa the Wise", 17, 11, 10, 3)
Ruaa the Wise lost against Yogurt Stormy
         Fighter("Luminous Zira", 9, 13, 24, 5)
Luminous Zira won against Yogurt Stormy
         Fighter("Maximal Egg", 11, 15, 10, 8)
Luminous Zira lost against Maximal E

LoadError: BoundsError: attempt to access 100-element Vector{Fighter} at index [101]

In [159]:
a_i, b_i

(101, 68)